## Enabling Import of Custom Python Modules from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%reload_ext autoreload
%autoreload 2

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Foreground + Background & Mask Image Generation

In [ ]:
import zipfile

path_to_zip_file = "/content/gdrive/My Drive/EVA4-Workspace/persons_fg/gen_fg_bg.zip"
directory_to_extract_to = '/content/'
print("Extracting zip file: %s" % path_to_zip_file)
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)
print("Extracted at: %s" % directory_to_extract_to)

In [ ]:
import zipfile

path_to_zip_file = "/content/fg_bg.zip"
directory_to_extract_to = '/content/'
print("Extracting zip file: %s" % path_to_zip_file)
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)
print("Extracted at: %s" % directory_to_extract_to)

Extracting zip file: /content/fg_bg.zip
Extracted at: /content/


Utils to process raw data downloaded from the internet


*   Resize bg into 224*224
*   Rescale fg and fg_mask into h=105 and scale down width accordingly
*   Rename the data

This was executed locally and the processed data was stored in the drive as "**gen_fg_bg.zip**"


In [ ]:
import os, sys
from PIL import Image
import numpy as np

def sizes(l):
  l.sort()
  sizes = []
  for infile in l:
    try:
      im = Image.open(infile)
      print(infile, im.size)
      sizes.append([infile, im.size])
    except Exception as e:
      print("Err: %s" % e)
  return sizes

def thumbnail(l, fmt="JPEG"):
  l.sort()
  size = 105, 105
  for infile in l:
    try:
      im = Image.open(infile)
      base_w, base_h = im.size
      new_h = 105
      hpercent = float(base_h)/float(new_h)
      new_w = int(float(base_w)*float(hpercent))
      im.thumbnail((new_w, new_h), Image.ANTIALIAS)
      im.save(infile, fmt)
    except IOError:
      print("cannot create thumbnail for '%s'" % infile)

def rename(l, tag, ext="jpg"):
  l.sort()
  for idx, src in enumerate(l):
    dst = f"{tag}_{idx:02d}.{ext}"
    os.rename(src, dst)

def reshape(l):
  size = 224, 224
  for infile in l:
    try:
      im = Image.open(infile)
      im = im.resize(size, Image.LANCZOS)
      im.save(infile, "JPEG")
    except IOError:
      print("cannot resize '%s'" % infile)

def preprocess():
  base_dir = os.getcwd()

  mdir = f"{base_dir}/bg/"
  os.chdir(mdir)
  l = os.listdir(".")
  # sm = sizes(l)
  reshape(l)
  rename(l, "bg", "jpg")

  mdir = f"{base_dir}/fg_mask/"
  os.chdir(mdir)
  l = os.listdir(".")
  # sm = sizes(l)
  thumbnail(l, "JPEG")
  rename(l, "fg", "jpg")

  mdir = f"{base_dir}/fg/"
  os.chdir(mdir)
  l = os.listdir(".")
  # sf = sizes(l)
  thumbnail(l, "PNG")
  rename(l, "fg", "png")

Utils to overlay foreground on background and create its mask also


*   Each foreground is flipped horizontally
*   For each orientation of foreground, randomly overlay it on the background 20 times.
*   #bg = 100
*   #fg = 100
*   #fg_bg = 100\*100\*2\*20 = 400,000



In [ ]:
from zipfile import ZipFile, ZIP_STORED
import io

def overlay_fg_on_bg():
  bg_dir = "./bg/"
  bg_imgs = sorted(os.listdir(bg_dir))
  fg_dir = "./fg/"
  fg_imgs = sorted(os.listdir(fg_dir))
  fg_masks_dir = "./fg_mask/"
  assert(len(bg_imgs) == 100)
  assert(len(fg_imgs) == 100)
  for bidx, bg_img in enumerate(bg_imgs):
    out_zip = ZipFile('fg_bg.zip', mode='a', compression=ZIP_STORED)
    label_info = open(f'fg_bg_labels.txt', 'w+')
    idx = 4000 * bidx
    print(f'Processing BG {bidx}')
    mbg = Image.open(bg_dir + bg_img)
    for fidx, fg_img in enumerate(fg_imgs):
      mfg = Image.open(fg_dir + fg_img)
      assert(fg_img[-4:] == ".png")
      mfg_mask = Image.open(fg_masks_dir + fg_img[:-4] + ".jpg")

      for i in range(20):
        for should_flip in [True, False]:
          bg = mbg.copy()
          fg = mfg.copy()
          fg_mask = mfg_mask.copy()

          if should_flip:
            fg = fg.transpose(Image.FLIP_LEFT_RIGHT)
            fg_mask = fg_mask.transpose(Image.FLIP_LEFT_RIGHT)

            bg_w, bg_h = bg.size
            fg_w, fg_h = fg.size
            max_h = bg_h - fg_h
            max_w = bg_w - fg_w
            pos_x = np.random.randint(low=0, high=max_w, size=1)[0]
            pos_y = np.random.randint(low=0, high=max_h, size=1)[0]

            bg.paste(fg, (pos_x, pos_y), fg)

            bg_mask = Image.new('L', bg.size)
            fg_mask = fg_mask.convert('L')
            bg_mask.paste(fg_mask, (pos_x, pos_y), fg_mask)

            bg.save('b_temp.jpg', optimize=True, quality=65)
            bg_mask.save('m_temp.jpg', optimize=True, quality=65)

            out_zip.write('b_temp.jpg', f'fg_bg/{bg_img[:-4]}/{bg_img[:-4]}_{fg_img[:-4]}_{"T" if should_flip else "F"}_{idx:06d}.jpg')
            out_zip.write('m_temp.jpg', f'fg_bg_masks/{bg_img[:-4]}/{bg_img[:-4]}_{fg_img[:-4]}_{"T" if should_flip else "F"}_{idx:06d}.jpg')
            label_info.write(f'{bg_img[:-4]}_{fg_img[:-4]}_{"T" if should_flip else "F"}_{idx:06d}.jpg\t{pos_x}\t{pos_y}\n')

            idx += 1
		
    label_info.close()
    out_zip.write('fg_bg_labels.txt', f'labels/{bg_img[:-4]}_label_info.txt')
    out_zip.close()


Generate the fg_bg images and masks

In [ ]:
overlay_fg_on_bg()

Processing BG 0
Processing BG 1
Processing BG 2
Processing BG 3
Processing BG 4
Processing BG 5
Processing BG 6
Processing BG 7
Processing BG 8
Processing BG 9
Processing BG 10
Processing BG 11
Processing BG 12
Processing BG 13
Processing BG 14
Processing BG 15
Processing BG 16
Processing BG 17
Processing BG 18
Processing BG 19
Processing BG 20
Processing BG 21
Processing BG 22
Processing BG 23
Processing BG 24
Processing BG 25
Processing BG 26
Processing BG 27
Processing BG 28
Processing BG 29
Processing BG 30
Processing BG 31
Processing BG 32
Processing BG 33
Processing BG 34
Processing BG 35
Processing BG 36
Processing BG 37
Processing BG 38
Processing BG 39
Processing BG 40
Processing BG 41
Processing BG 42
Processing BG 43
Processing BG 44
Processing BG 45
Processing BG 46
Processing BG 47
Processing BG 48
Processing BG 49
Processing BG 50
Processing BG 51
Processing BG 52
Processing BG 53
Processing BG 54
Processing BG 55
Processing BG 56
Processing BG 57
Processing BG 58
Process

## Depth Image Generation

In [ ]:
import zipfile

path_to_zip_file = "/content/gdrive/My Drive/fg_bg_7z.zip"
directory_to_extract_to = '/content/'
print("Extracting zip file: %s" % path_to_zip_file)
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  for f in zip_ref.namelist():
    if f.startswith('fg_bg/'):
      zip_ref.extract(f, directory_to_extract_to)
print("Extracted at: %s" % directory_to_extract_to)

Extracting zip file: /content/gdrive/My Drive/fg_bg_7z.zip
Extracted at: /content/


In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-6ea1e9e2-ae46-586f-520f-39ef8321424b)


In [ ]:
#!git clone https://github.com/csharpshooter/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 234, done.
remote: Total 234 (delta 0), reused 0 (delta 0), pack-reused 234
Receiving objects: 100% (234/234), 11.79 MiB | 6.05 MiB/s, done.
Resolving deltas: 100% (115/115), done.


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/DenseDepth-master')
import os
os.chdir('/content/gdrive/My Drive/DenseDepth-master')
os.getcwd()

'/content/gdrive/My Drive/DenseDepth-master'

In [ ]:
#!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

In [ ]:
from mytest1 import get_model, infer

Using TensorFlow backend.


In [ ]:
model = get_model()

Loading model...

Model loaded (nyu.h5).


In [ ]:
from zipfile import ZipFile, ZIP_STORED
import os
import zipfile
import datetime

bg_start = 90
bg_end = 100
#zip_file_name = f"fg_bg_depth_s{bg_start:02d}_e{bg_end:02d}.zip"
zip_file_name = 'fg_bg_depth_s56_e69.zip'

for bg_id in range(bg_start, bg_end):
    st = datetime.datetime.now()
    zipf = ZipFile(zip_file_name, mode='a', compression=ZIP_STORED)
    infer(model, bg_id, zipf=None, show_viz=False,
          batch_size=512, scale=float(7/3), base_path="/content/fg_bg")
    #zipf.close()
    et = datetime.datetime.now()
    delta = et-st
    print("processing time: ", delta)


Loading images from /content/fg_bg/bg192_90_*.jpg
Batch[1]: Loaded (512) images of size (448, 448, 3).
Creating archive: /content/gdrive/My Drive/fg_bg_depth.zip
Batch[2]: Loaded (512) images of size (448, 448, 3).
Batch[3]: Loaded (512) images of size (448, 448, 3).
Batch[4]: Loaded (512) images of size (448, 448, 3).
Batch[5]: Loaded (512) images of size (448, 448, 3).
Batch[6]: Loaded (512) images of size (448, 448, 3).
Batch[7]: Loaded (512) images of size (448, 448, 3).
Batch[8]: Loaded (416) images of size (448, 448, 3).
processing time:  0:13:26.229864

Loading images from /content/fg_bg/bg192_91_*.jpg
Batch[1]: Loaded (512) images of size (448, 448, 3).
Creating archive: /content/gdrive/My Drive/fg_bg_depth.zip
Batch[2]: Loaded (512) images of size (448, 448, 3).
Batch[3]: Loaded (512) images of size (448, 448, 3).
Batch[4]: Loaded (512) images of size (448, 448, 3).
Batch[5]: Loaded (512) images of size (448, 448, 3).
Batch[6]: Loaded (512) images of size (448, 448, 3).
Batch